In [1]:
from warnings import simplefilter 
simplefilter(action='ignore', category=FutureWarning)
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera

import argparse
import json
import cv2
import numpy as np

from yolo.frontend import create_yolo
from yolo.backend.utils.box import draw_scaled_boxes
from yolo.backend.utils.annotation import parse_annotation
from yolo.backend.utils.eval.fscore import count_true_positives, calc_score
import yolo.backend.utils.box as box
from pascal_voc_writer import Writer
from shutil import copyfile
import os
import yolo
import time


DEFAULT_CONFIG_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "config.json")
DEFAULT_WEIGHT_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "weights.h5")
DEFAULT_THRESHOLD = 0.3


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [2]:


with open('turn4/config.json') as config_buffer:
    config = json.loads(config_buffer.read())

# 2. create yolo instance & predict
yolo = create_yolo(config['model']['architecture'],
                   config['model']['labels'],
                   config['model']['input_size'],
                   config['model']['anchors'])
yolo.load_weights('model.h5')



Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenet_0.75_224 (Model)   (None, 7, 7, 768)         1832976   
_________________________________________________________________
detection_layer_50 (Conv2D)  (None, 7, 7, 50)          38450     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 5, 10)       0         
Total params: 1,871,426
Trainable params: 1,855,010
Non-trainable params: 16,416
_________________________________________________________________
Loading pre-trained weights in model.h5


In [3]:

from IPython.display import clear_output

In [5]:
camera = CSICamera(width=320, height=240, capture_fps=120)

camera.running = True

In [6]:
image = camera.value
image_widget = ipywidgets.Image(format='jpeg')
label_widget = ipywidgets.Label()
image_widget.value = bgr8_to_jpeg(image)

display(image_widget, label_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Label(value='')

In [ ]:
timer = 0
timer2 = 0
while 1 :
    timer2 = time.time()
    image  = camera.value
    image = image[8:8+224, 48:48+224]
    boxes, probs = yolo.predict(image, float(0.6))
    if len(boxes) == 0 :
#         image = box.draw_scaled_boxes(image, boxes, probs, config['model']['labels'])
        image_widget.value = bgr8_to_jpeg(image)
#         label_widget.value = "frame rates: " + str(int(1/timer)) + " fps"
        continue
    tong2 = box.to_centroid(boxes)
    # labels = np.argmax(probs, axis=1)
    # if len(probs) > 0 else [] 
    timer = time.time() - timer2
    
    image = cv2.circle(image, (int(tong2[0][0]),int(tong2[0][1])), 3, (255, 0, 0), 3)
    
    image = box.draw_boxes(image, boxes, probs, config['model']['labels'])
    image_widget.value = bgr8_to_jpeg(image)
    label_widget.value = "frame rates: " + str(int(1/timer)) + " fps"
#     clear_output()
#     print(tong2)
    


In [ ]:
display(label)